# 1. Imports, Options and Ingestion

In [ ]:
# Imports list
import joblib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neighbors import LocalOutlierFactor
from sklearn.decomposition import PCA

from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.metrics import silhouette_score

from minisom import MiniSom

from matplotlib.patches import RegularPolygon, Ellipse
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import cm, colorbar
from matplotlib import colors as mpl_colors
from matplotlib.colors import LinearSegmentedColormap

from matplotlib.lines import Line2D
import seaborn as sns

from matplotlib import __version__ as mplver

from sklearn.impute import KNNImputer
from sklearn.metrics import root_mean_squared_error

In [ ]:
# Style is importantuntitled:Untitled-1.ipynb?jupyter-notebook
sns.set(style="white")

# Ensuring pandas always prints all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)
pd.set_option('display.max_colwidth', 1000)

%config InlineBackend.figure_format = 'retina'

In [ ]:
path = '...'

In [ ]:
# Reading the csv data
data = pd.read_csv(f'{path}all_customers_preproc.csv', index_col=0)

regulars = pd.read_csv(f'{path}regulars_preproc.csv', index_col=0)

In [49]:
regulars.head()

,cust_age,n_vendor,n_product,n_chain,first_order,last_order,american,asian,beverages,cafe,chicken_dishes,chinese,desserts,healthy,indian,italian,japanese,noodle_dishes,other,street_food_snacks,thai,DOW_0,DOW_1,DOW_2,DOW_3,DOW_4,DOW_5,DOW_6,HR_0,HR_1,HR_2,HR_3,HR_4,HR_5,HR_6,HR_7,HR_8,HR_9,HR_10,HR_11,HR_12,HR_13,HR_14,HR_15,HR_16,HR_17,HR_18,HR_19,HR_20,HR_21,HR_22,HR_23,total_amt,n_order,avg_amt_per_product,avg_amt_per_order,avg_amt_per_vendor,days_cust,avg_days_to_order,days_due,per_chain_order,n_days_week,n_times_day,n_cuisines,log_n_vendor,log_n_product,log_n_chain,log_american,log_asian,log_beverages,log_cafe,log_chicken_dishes,log_chinese,log_desserts,log_healthy,log_indian,log_italian,log_japanese,log_noodle_dishes,log_other,log_street_food_snacks,log_thai,log_total_amt,log_n_order,log_avg_amt_per_product,log_avg_amt_per_order,log_avg_amt_per_vendor,log_n_days_week,log_n_times_day,avg_amt_per_day,avg_product_per_day,avg_order_per_day,cust_region_2360.0,cust_region_2400.0,cust_region_4140.0,cust_region_4660.0,cust_region_8370.0,cust_region_8550.0,cust_region_8670.0,last_promo_DELIVERY,last_promo_DISCOUNT,last_promo_FREEBIE,last_promo_NO_PROMO,pay_method_CARD,pay_method_CASH,pay_method_DIGI,cust_city_2.0,cust_city_4.0,cust_city_8.0,regular,foodie_flag,gluttonous_flag,loyal_flag,top_cuisine,transaction_volume,interaction_rate
customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
118371798366,-1.345900,-0.622187,-0.252742,-0.633329,-1.131842,-3.500568,-0.486170,-0.488927,-0.317568,-0.165088,-0.273312,-0.260873,-0.20627,-0.205561,4.061113,-0.342266,-0.363981,-0.202848,-0.372285,-0.278876,-0.230057,0.356957,-0.646944,-0.674014,-0.689695,-0.73492,-0.705798,0.160186,-0.174681,-0.189373,-0.200666,-0.257385,-0.25538,-0.247911,-0.231751,-0.24215,-0.267549,-0.367063,-0.428853,-0.464903,-0.433021,-0.420768,-0.407075,-0.431283,-0.464805,-0.469156,1.763653,-0.362839,-0.292908,-0.22833,-0.182616,-0.186099,-0.332279,-0.700352,-0.337798,0.899607,0.162319,-1.759441,-1.443647,3.025425,-0.387035,-0.792966,-1.213175,-1.120173,-0.701528,-0.033868,-0.729523,-0.739557,-0.784371,-0.459818,-0.218988,-0.348461,-0.360903,-0.262785,-0.277907,3.743945,-0.504522,-0.511543,-0.277011,-0.545779,-0.389082,-0.29454,-0.053734,-1.020872,-0.129327,1.050632,0.505462,-0.794926,-1.783859,7.819311,12.574166,6.402312,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,True,0,0,0,indian,15.549184,-0.102906
400089128395,-1.487186,-0.622187,-0.769539,-0.356817,-1.131842,-3.500568,0.625611,-0.211512,-0.317568,-0.165088,-0.273312,-0.260873,-0.20627,-0.205561,-0.256206,-0.342266,-0.363981,-0.202848,-0.372285,-0.278876,-0.230057,0.356957,-0.646944,-0.674014,-0.689695,-0.73492,-0.705798,0.160186,-0.174681,-0.189373,-0.200666,-0.257385,-0.25538,-0.247911,-0.231751,-0.24215,-0.267549,-0.367063,0.706668,0.598445,-0.433021,-0.420768,-0.407075,-0.431283,-0.464805,-0.469156,-0.432474,-0.362839,-0.292908,-0.22833,-0.182616,-0.186099,-0.573930,-0.700352,0.825949,0.086764,-0.313801,-1.759441,-1.443647,3.025425,1.032029,-0.792966,-0.711543,-0.431058,-0.701528,-1.214069,-0.149593,1.329052,0.503190,-0.459818,-0.218988,-0.348461,-0.360903,-0.262785,-0.277907,-0.351143,-0.504522,-0.511543,-0.277011,-0.545779,-0.389082,-0.29454,-0.542400,-1.020872,0.874691,0.367706,-0.086075,-0.794926,-0.763012,5.042618,4.670481,6.402312,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,True,0,0,0,american,9.334548,-0.735079
1060080106083,1.479818,-1.003993,-0.769539,-0.356817,-1.131842,-3.500568,0.311676,-0.488927,-0.317568,-0.165088,-0.273312,-0.260873,-0.20627,-0.205561,-0.256206,-0.342266,-0.363981,-0.202848,-0.372285,-0.278876,-0.230057,0.356957,-0.646944,-0.674014,-0.689695,-0.73492,-0.705798,0.160186,-0.174681,-0.189373,-0.200666,-0.257385,-0.25538,-0.247911,-0.231751,-0.24215,-0.267549,1.007813,-0.428853,0.598445,-0.433021,-0.420768,-0.407075,-0.431283,-0.464805,-0.469156,-0.432474,-0.3628